# Wrangle Flight Data 2008

## Author Ram Saran Vuppuluri.

In this project we are going to analyze:

1.	On-ground performance of various airlines and airports operating in United States during 2008. 
2.	Connectivity map and airline frequency.
3.	Cause of delay.

We need perform following data wrangling steps before doing any analysis.

1.	Gather data
2.	Assess data
3.	Clean data

## Gather Data

For this project we have manually downloaded __[2008.csv,airports.csv, carriers.csv](http://stat-computing.org/dataexpo/2009/the-data.html)__ and loaded the data into filghts_2008 data frame.



In [1]:
import pandas as pd
import datetime

In [2]:
flights_2008 = pd.read_csv("2008.csv")

## Assess Data

We will perform visual and programmatic assessment of data in filghts_2008 data frame.

### Visual Assessment

From below visual assessment we find:

1.	Following columns are loaded as float64 instead of Time
    * DepTime
    * ArrTime
2.	Following columns are loaded as int64 instead of Time
    * CRSDepTime
    * CRSArrTime
3.	Year, Month and Day of Month are separate int64 attributes instead of Date.
4.	Following columns are loaded as int64 instead of Boolean
    * Cancelled
    * Diverted
5.	We will create a new column with Boolean Delayed, which will populate if flight is delayed.

    __Note__: In this case we cannot restructure delay time into one column because single delay can be caused by more than one reason.
6.	We will create a new column Route of type object which is a Origin + “-“ + Dest.
7. Change Carrier, Origin and Dest to Categorical.

In [3]:
flights_2008.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 1.5+ GB


### Programmatic Assessment

From below programmatic assessment we find:

1.	Following columns have nan values:
    * DepTime – Missing for cancelled flights
    * ArrTime  – Missing for cancelled or diverted flights
    * TailNum – Missing for cancelled or diverted flights. There are 5 records with no tail number. We are not going to utilize this column in our analysis.
    * ActualElapsedTime – Missing for cancelled or diverted flights
    * CRSElapsedTime – Missing for cancelled or diverted flights
    * AirTime  – Missing for cancelled or diverted flights
    * ArrDelay – Missing for cancelled or diverted flights
    * DepDelay – Missing for cancelled or diverted flights
    * TaxiIn – Missing for cancelled or diverted flights
    * TaxiOut – Missing for cancelled or diverted flights
    * CancellationCode – Missing for non-cancelled flight
    * CarrierDelay – Missing for non-delayed flights
    * WeatherDelay – Missing for non-delayed flights
    * NASDelay – Missing for non-delayed flights
    * SecurityDelay – Missing for non-delayed flights
    * LateAircraftDelay – Missing for non-delayed flights

In [4]:
flights_2008.isna().any()

Year                 False
Month                False
DayofMonth           False
DayOfWeek            False
DepTime               True
CRSDepTime           False
ArrTime               True
CRSArrTime           False
UniqueCarrier        False
FlightNum            False
TailNum               True
ActualElapsedTime     True
CRSElapsedTime        True
AirTime               True
ArrDelay              True
DepDelay              True
Origin               False
Dest                 False
Distance             False
TaxiIn                True
TaxiOut               True
Cancelled            False
CancellationCode      True
Diverted             False
CarrierDelay          True
WeatherDelay          True
NASDelay              True
SecurityDelay         True
LateAircraftDelay     True
dtype: bool

In [5]:
len(flights_2008[(flights_2008.DepTime.isna())])

136246

In [6]:
len(flights_2008[(flights_2008.DepTime.isna()) & ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

136246

In [7]:
len(flights_2008[(flights_2008.ArrTime.isna())])

151649

In [8]:
len(flights_2008[(flights_2008.ArrTime.isna()) & ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

151649

In [9]:
len(flights_2008[(flights_2008.TailNum.isna())])

83365

In [10]:
len(flights_2008[(flights_2008.TailNum.isna()) & ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

83360

In [11]:
flights_2008[(flights_2008.TailNum.isna()) & ((flights_2008.Cancelled == 0) & (flights_2008.Diverted == 0))]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1643793,2008,3,18,2,1716.0,1655,2036.0,1947,9E,3760,...,11.0,29.0,0,NaN,0,0.0,0.0,28.0,0.0,21.0
1656781,2008,3,10,1,853.0,852,951.0,953,9E,5889,...,3.0,30.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2543429,2008,5,8,4,1545.0,1530,1711.0,1707,OH,5043,...,5.0,15.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2556354,2008,5,31,6,1720.0,1244,1920.0,1427,OH,5396,...,8.0,40.0,0,NaN,0,0.0,276.0,17.0,0.0,0.0
6616551,2008,12,31,3,830.0,835,1023.0,1015,OH,6624,...,8.0,13.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [12]:
len(flights_2008[flights_2008.ActualElapsedTime.isna()])

154699

In [13]:
len(flights_2008[(flights_2008.ActualElapsedTime.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

154699

In [14]:
len(flights_2008[flights_2008.AirTime.isna()])

154699

In [15]:
len(flights_2008[(flights_2008.AirTime.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

154699

In [16]:
len(flights_2008[flights_2008.ArrDelay.isna()])

154699

In [17]:
len(flights_2008[(flights_2008.ArrDelay.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

154699

In [18]:
len(flights_2008[flights_2008.DepDelay.isna()])

136246

In [19]:
len(flights_2008[(flights_2008.DepDelay.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

136246

In [20]:
len(flights_2008[flights_2008.TaxiIn.isna()])

151649

In [21]:
len(flights_2008[(flights_2008.TaxiIn.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

151649

In [22]:
len(flights_2008[flights_2008.TaxiOut.isna()])

137058

In [23]:
len(flights_2008[(flights_2008.TaxiOut.isna())& ((flights_2008.Cancelled == 1) | (flights_2008.Diverted == 1))])

137058

In [24]:
len(flights_2008[flights_2008.CancellationCode.isna()])

6872294

In [25]:
len(flights_2008[(flights_2008.CancellationCode.isna())& (flights_2008.Cancelled == 0)])

6872294

In [26]:
len(flights_2008[flights_2008.CarrierDelay.isna()])

5484993

In [27]:
len(flights_2008[flights_2008.WeatherDelay.isna()])

5484993

In [28]:
len(flights_2008[flights_2008.NASDelay.isna()])

5484993

In [29]:
len(flights_2008[flights_2008.SecurityDelay.isna()])

5484993

In [30]:
len(flights_2008[flights_2008.LateAircraftDelay.isna()])

5484993

From the above visual and programmatic assessment, we find following quality and tidiness issues.

### Quality issues:

1. Following columns have nan values:
    * DepTime – Missing for cancelled flights
    * ArrTime  – Missing for cancelled or diverted flights
    * ActualElapsedTime – Missing for cancelled or diverted flights
    * CRSElapsedTime – Missing for cancelled or diverted flights
    * AirTime  – Missing for cancelled or diverted flights
    * ArrDelay – Missing for cancelled or diverted flights
    * DepDelay – Missing for cancelled or diverted flights
    * TaxiIn – Missing for cancelled or diverted flights
    * TaxiOut – Missing for cancelled or diverted flights
    * CancellationCode – Missing for non-cancelled flight
    * CarrierDelay – Missing for non-delayed flights
    * WeatherDelay – Missing for non-delayed flights
    * NASDelay – Missing for non-delayed flights
    * SecurityDelay – Missing for non-delayed flights
    * LateAircraftDelay – Missing for non-delayed flights
2. Following columns are loaded as float64 instead of Time
    * DepTime
    * ArrTime
3. Following columns are loaded as int64 instead of Time
    * CRSDepTime
    * CRSArrTime
4. Following columns are loaded as int64 instead of Boolean
    * Cancelled
    * Diverted
5. Change Carrier, Origin and Dest to Categorical.

### Tidiness issues:

1. Year, Month and Day of Month are separate int64 attributes instead of Date. We will delete individual fields and create consolidated date time columns for CRSDep, CRSArr, Dep and Arr. There are some time fields with value 24:00 which should change date to next date.
2.	We will create a new column with Boolean Delayed, which will populate if flight is delayed.

    __Note__: In this case we cannot restructure delay time into one column because single delay can be caused by more than one reason.
3.	We will create a new column Route of type object which is a Origin + “-“ + Dest.
4. TailNum column will be deleted as we are not going to utilize this column in our analysis.

## Clean Data

__Note__ as the dataframe memory foot print is large we are not going to create a new clean copy.

### Quality issues:

1. Following columns have nan values:
    * DepTime – Missing for cancelled flights
    * ArrTime  – Missing for cancelled or diverted flights
    * ActualElapsedTime – Missing for cancelled or diverted flights
    * CRSElapsedTime – Missing for cancelled or diverted flights
    * AirTime  – Missing for cancelled or diverted flights
    * ArrDelay – Missing for cancelled or diverted flights
    * DepDelay – Missing for cancelled or diverted flights
    * TaxiIn – Missing for cancelled or diverted flights
    * TaxiOut – Missing for cancelled or diverted flights
    * CancellationCode – Missing for non-cancelled flight
    * CarrierDelay – Missing for non-delayed flights
    * WeatherDelay – Missing for non-delayed flights
    * NASDelay – Missing for non-delayed flights
    * SecurityDelay – Missing for non-delayed flights
    * LateAircraftDelay – Missing for non-delayed flights
2. Following columns are loaded as float64 instead of Time
    * DepTime
    * ArrTime
3. Following columns are loaded as int64 instead of Time
    * CRSDepTime
    * CRSArrTime
4. Following columns are loaded as int64 instead of Boolean
    * Cancelled
    * Diverted
5. Change Carrier, Origin and Dest to Categorical.

### Code

DepTime – Missing for cancelled flights

In [31]:
flights_2008.DepTime.dtype

dtype('float64')

In [32]:
flights_2008.DepTime.fillna(value=0.0,inplace=True)

### Test

DepTime – Missing for cancelled flights

In [33]:
len(flights_2008[flights_2008.DepTime.isna()])

0

### Code

ArrTime  – Missing for cancelled or diverted flights

In [34]:
flights_2008.ArrTime.dtype

dtype('float64')

In [35]:
flights_2008.ArrTime.fillna(value=0.0,inplace=True)

### Test

ArrTime  – Missing for cancelled or diverted flights

In [36]:
len(flights_2008[flights_2008.ArrTime.isna()])

0

### Code

ActualElapsedTime – Missing for cancelled or diverted flights

In [37]:
flights_2008.ActualElapsedTime.dtype

dtype('float64')

In [38]:
flights_2008.ActualElapsedTime.fillna(value=0.0,inplace=True)

### Test

ActualElapsedTime – Missing for cancelled or diverted flights

In [39]:
len(flights_2008[flights_2008.ActualElapsedTime.isna()])

0

### Code

CRSElapsedTime – Missing for cancelled or diverted flights

In [40]:
flights_2008.CRSElapsedTime.dtype

dtype('float64')

In [41]:
flights_2008.CRSElapsedTime.fillna(value=0.0,inplace=True)

### Test

CRSElapsedTime – Missing for cancelled or diverted flights

In [42]:
len(flights_2008[flights_2008.CRSElapsedTime.isna()])

0

### Code

AirTime  – Missing for cancelled or diverted flights

In [43]:
flights_2008.AirTime.dtype

dtype('float64')

In [44]:
flights_2008.AirTime.fillna(value=0.0,inplace=True)

### Test

AirTime  – Missing for cancelled or diverted flights

In [45]:
len(flights_2008[flights_2008.AirTime.isna()])

0

### Code

ArrDelay – Missing for cancelled or diverted flight

In [46]:
flights_2008.ArrDelay.dtype

dtype('float64')

In [47]:
flights_2008.ArrDelay.fillna(value=0.0,inplace=True)

### Test

ArrDelay – Missing for cancelled or diverted flight

In [48]:
len(flights_2008[flights_2008.ArrDelay.isna()])

0

### Code

DepDelay – Missing for cancelled or diverted flights

In [49]:
flights_2008.DepDelay.dtype

dtype('float64')

In [50]:
flights_2008.DepDelay.fillna(value=0.0,inplace=True)

### Test

DepDelay – Missing for cancelled or diverted flights

In [51]:
len(flights_2008[flights_2008.DepDelay.isna()])

0

### Code

TaxiIn – Missing for cancelled or diverted flights

In [52]:
flights_2008.TaxiIn.dtype

dtype('float64')

In [53]:
flights_2008.TaxiIn.fillna(value=0.0,inplace=True)

### Test

TaxiIn – Missing for cancelled or diverted flights

In [54]:
len(flights_2008[flights_2008.TaxiIn.isna()])

0

### Code

TaxiOut – Missing for cancelled or diverted flights

In [55]:
flights_2008.TaxiOut.dtype

dtype('float64')

In [56]:
flights_2008.TaxiOut.fillna(value=0.0,inplace=True)

### Test

TaxiOut – Missing for cancelled or diverted flights

In [57]:
len(flights_2008[flights_2008.TaxiOut.isna()])

0

### Code

CancellationCode – Missing for non-cancelled flights

In [58]:
flights_2008.CancellationCode.dtype

dtype('O')

In [59]:
flights_2008.CancellationCode.fillna(value='',inplace=True)

### Test

CancellationCode – Missing for non-cancelled flights

In [60]:
len(flights_2008[flights_2008.CancellationCode.isna()])

0

### Code

CarrierDelay – Missing for non-delayed flights

In [61]:
flights_2008.CarrierDelay.dtype

dtype('float64')

In [62]:
flights_2008.CarrierDelay.fillna(value=0.0,inplace=True)

### Test

CarrierDelay – Missing for non-delayed flights

In [63]:
len(flights_2008[flights_2008.CarrierDelay.isna()])

0

### Code

WeatherDelay – Missing for non-delayed flights

In [64]:
flights_2008.WeatherDelay.dtype

dtype('float64')

In [65]:
flights_2008.WeatherDelay.fillna(value=0.0,inplace=True)

### Test

WeatherDelay – Missing for non-delayed flights

In [66]:
len(flights_2008[flights_2008.WeatherDelay.isna()])

0

### Code

NASDelay – Missing for non-delayed flights

In [67]:
flights_2008.NASDelay.dtype

dtype('float64')

In [68]:
flights_2008.NASDelay.fillna(value=0.0,inplace=True)

### Test

NASDelay – Missing for non-delayed flights

In [69]:
len(flights_2008[flights_2008.NASDelay.isna()])

0

### Code

SecurityDelay – Missing for non-delayed flights

In [70]:
flights_2008.SecurityDelay.dtype

dtype('float64')

In [71]:
flights_2008.SecurityDelay.fillna(value=0.0,inplace=True)

### Test

SecurityDelay – Missing for non-delayed flights

In [72]:
len(flights_2008[flights_2008.SecurityDelay.isna()])

0

### Code

LateAircraftDelay – Missing for non-delayed flights

In [73]:
flights_2008.LateAircraftDelay.dtype

dtype('float64')

In [74]:
flights_2008.LateAircraftDelay.fillna(value=0.0,inplace=True)

### Test

LateAircraftDelay – Missing for non-delayed flights

In [75]:
len(flights_2008[flights_2008.LateAircraftDelay.isna()])

0

### Test

Now that we have cleaned identified columns that contain NAN value, below code should return False for all columns except "TailNum" which we are going to delete later in the cleaning process.

In [76]:
flights_2008.isna().any()

Year                 False
Month                False
DayofMonth           False
DayOfWeek            False
DepTime              False
CRSDepTime           False
ArrTime              False
CRSArrTime           False
UniqueCarrier        False
FlightNum            False
TailNum               True
ActualElapsedTime    False
CRSElapsedTime       False
AirTime              False
ArrDelay             False
DepDelay             False
Origin               False
Dest                 False
Distance             False
TaxiIn               False
TaxiOut              False
Cancelled            False
CancellationCode     False
Diverted             False
CarrierDelay         False
WeatherDelay         False
NASDelay             False
SecurityDelay        False
LateAircraftDelay    False
dtype: bool

### Code
Below code also covers

### Quality issues:
2. Following columns are loaded as float64 instead of Time
    * DepTime
    * ArrTime
3. Following columns are loaded as int64 instead of Time
    * CRSDepTime
    * CRSArrTime
    
### Tidiness issues:

1. Year, Month and Day of Month are separate int64 attributes instead of Date. We will delete individual fields and create consolidated date time columns for CRSDep, CRSArr, Dep and Arr. There are some time fields with value 24:00 which should change date to next date.

4. TailNum column will be deleted as we are not going to utilize this column in our analysis.

In [77]:
flights_2008['DepTime']=flights_2008.DepTime.astype(int).astype(str)

In [78]:
flights_2008.DepTime.dtype

dtype('O')

In [79]:
len(flights_2008[(flights_2008.DepTime.str.len() < 4)])

2083706

In [80]:
flights_2008['DepTime']=flights_2008.DepTime.str.rjust(4,'0')

In [81]:
len(flights_2008[(flights_2008.DepTime.str.len() < 4)])

0

In [82]:
flights_2008['DepTime']=flights_2008.DepTime.str[:2]+":"+flights_2008.DepTime.str[2:]

In [83]:
flights_2008['ArrTime']=flights_2008.ArrTime.astype(int).astype(str)

In [84]:
flights_2008.ArrTime.dtype

dtype('O')

In [85]:
len(flights_2008[(flights_2008.ArrTime.str.len() < 4)])

1373761

In [86]:
flights_2008['ArrTime']=flights_2008.ArrTime.str.rjust(4,'0')

In [87]:
len(flights_2008[(flights_2008.ArrTime.str.len() < 4)])

0

In [88]:
flights_2008['ArrTime']=flights_2008.ArrTime.str[:2]+":"+flights_2008.ArrTime.str[2:]

In [89]:
flights_2008['CRSDepTime']=flights_2008.CRSDepTime.astype(str)

In [90]:
flights_2008.CRSDepTime.dtype

dtype('O')

In [91]:
len(flights_2008[(flights_2008.CRSDepTime.str.len() < 4)])

1987593

In [92]:
flights_2008['CRSDepTime']=flights_2008.CRSDepTime.str.rjust(4,'0')

In [93]:
len(flights_2008[(flights_2008.CRSDepTime.str.len() < 4)])

0

In [94]:
flights_2008['CRSDepTime']=flights_2008.CRSDepTime.str[:2]+":"+flights_2008.CRSDepTime.str[2:]

In [95]:
flights_2008['CRSArrTime']=flights_2008.CRSArrTime.astype(str)

In [96]:
flights_2008.CRSArrTime.dtype

dtype('O')

In [97]:
len(flights_2008[(flights_2008.CRSArrTime.str.len() < 4)])

1169008

In [98]:
flights_2008['CRSArrTime']=flights_2008.CRSArrTime.str.rjust(4,'0')

In [99]:
len(flights_2008[(flights_2008.CRSArrTime.str.len() < 4)])

0

In [100]:
flights_2008['CRSArrTime']=flights_2008.CRSArrTime.str[:2]+":"+flights_2008.CRSArrTime.str[2:]

In [101]:
len(flights_2008[flights_2008.DepTime=='24:00']),len(flights_2008[flights_2008.ArrTime=='24:00'])

(521, 2677)

In [102]:
len(flights_2008[flights_2008.CRSDepTime=='24:00']),len(flights_2008[flights_2008.CRSArrTime=='24:00'])

(0, 547)

In [103]:
flights_2008[(flights_2008.DepTime=='24:00')&(flights_2008.ArrTime=='24:00')&(flights_2008.CRSArrTime=='24:00')]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


In [104]:
flights_2008['DepTimeDate'] = pd.to_datetime(flights_2008.Year.astype(str) +"-"+flights_2008.Month.astype(str)+"-"+flights_2008.DayofMonth.astype(str))

In [105]:
flights_2008['ArrTimeDate'] = flights_2008['DepTimeDate']
flights_2008['CRSArrTimeDate'] = flights_2008['DepTimeDate']
flights_2008['CRSDepTimeDate'] = flights_2008['DepTimeDate']

In [106]:
flights_2008['DepTimeDate'] = flights_2008[flights_2008.DepTime=='24:00'].DepTimeDate+ datetime.timedelta(days=1)

In [107]:
len(flights_2008[flights_2008.DepTimeDate.isna()])

7009207

In [108]:
flights_2008.DepTimeDate.fillna(flights_2008.ArrTimeDate,inplace=True)

In [109]:
len(flights_2008[flights_2008.DepTimeDate.isna()])

0

In [110]:
flights_2008['ArrTimeDate'] =flights_2008[flights_2008.ArrTime=='24:00'].ArrTimeDate+ datetime.timedelta(days=1)

In [111]:
len(flights_2008[flights_2008.ArrTimeDate.isna()])

7007051

In [112]:
flights_2008.ArrTimeDate.fillna(flights_2008.DepTimeDate,inplace=True)

In [113]:
len(flights_2008[flights_2008.ArrTimeDate.isna()])

0

In [114]:
flights_2008['CRSArrTimeDate'] =flights_2008[flights_2008.CRSArrTime=='24:00'].CRSArrTimeDate+ datetime.timedelta(days=1)

In [115]:
len(flights_2008[flights_2008.CRSArrTimeDate.isna()])

7009181

In [116]:
flights_2008.CRSArrTimeDate.fillna(flights_2008.DepTimeDate,inplace=True)

In [117]:
len(flights_2008[flights_2008.CRSArrTimeDate.isna()])

0

In [118]:
flights_2008.DepTime.replace("24:00","00:00",inplace=True);
flights_2008.CRSDepTime.replace("24:00","00:00",inplace=True);
flights_2008.ArrTime.replace("24:00","00:00",inplace=True);
flights_2008.CRSArrTime.replace("24:00","00:00",inplace=True);

In [119]:
flights_2008['DepDateTime']=pd.to_datetime(flights_2008.DepTimeDate.astype(str)+" "+flights_2008.DepTime);

In [120]:
flights_2008['CRSDepDateTime']=pd.to_datetime(flights_2008.CRSDepTimeDate.astype(str)+" "+flights_2008.CRSDepTime);

In [121]:
flights_2008['ArrDateTime']=pd.to_datetime(flights_2008.ArrTimeDate.astype(str)+" "+flights_2008.ArrTime);

In [122]:
flights_2008['CRSArrDateTime']=pd.to_datetime(flights_2008.CRSArrTimeDate.astype(str)+" "+flights_2008.CRSArrTime);

In [123]:
col_del_list = ['Year','Month','DayofMonth','DepTime','CRSDepTime','ArrTime','CRSArrTime','TailNum','DepTimeDate','ArrTimeDate','CRSArrTimeDate','CRSDepTimeDate']

In [124]:
flights_2008.drop(col_del_list,axis=1,inplace=True)

## Test

In [125]:
flights_2008.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 25 columns):
DayOfWeek            int64
UniqueCarrier        object
FlightNum            int64
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
DepDateTime          datetime64[ns]
CRSDepDateTime       datetime64[ns]
ArrDateTime          datetime64[ns]
CRSArrDateTime       datetime64[ns]
dtypes: datetime64[ns](4), float64(12), int64(5), object(4)
memory usage: 1.3+ GB


In [126]:
flights_2008.isna().any()

DayOfWeek            False
UniqueCarrier        False
FlightNum            False
ActualElapsedTime    False
CRSElapsedTime       False
AirTime              False
ArrDelay             False
DepDelay             False
Origin               False
Dest                 False
Distance             False
TaxiIn               False
TaxiOut              False
Cancelled            False
CancellationCode     False
Diverted             False
CarrierDelay         False
WeatherDelay         False
NASDelay             False
SecurityDelay        False
LateAircraftDelay    False
DepDateTime          False
CRSDepDateTime       False
ArrDateTime          False
CRSArrDateTime       False
dtype: bool

In [127]:
flights_2008.head()

,DayOfWeek,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,...,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDateTime,CRSDepDateTime,ArrDateTime,CRSArrDateTime
0,4,WN,335,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,...,0,0.0,0.0,0.0,0.0,0.0,2008-01-03 20:03:00,2008-01-03 19:55:00,2008-01-03 22:11:00,2008-01-03 22:25:00
1,4,WN,3231,128.0,145.0,113.0,2.0,19.0,IAD,TPA,...,0,0.0,0.0,0.0,0.0,0.0,2008-01-03 07:54:00,2008-01-03 07:35:00,2008-01-03 10:02:00,2008-01-03 10:00:00
2,4,WN,448,96.0,90.0,76.0,14.0,8.0,IND,BWI,...,0,0.0,0.0,0.0,0.0,0.0,2008-01-03 06:28:00,2008-01-03 06:20:00,2008-01-03 08:04:00,2008-01-03 07:50:00
3,4,WN,1746,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,...,0,0.0,0.0,0.0,0.0,0.0,2008-01-03 09:26:00,2008-01-03 09:30:00,2008-01-03 10:54:00,2008-01-03 11:00:00
4,4,WN,3920,90.0,90.0,77.0,34.0,34.0,IND,BWI,...,0,2.0,0.0,0.0,0.0,32.0,2008-01-03 18:29:00,2008-01-03 17:55:00,2008-01-03 19:59:00,2008-01-03 19:25:00


## Code

### Quality issues:

4. Following columns are loaded as int64 instead of Boolean
    * Cancelled
    * Diverted


In [128]:
flights_2008.Cancelled.dtype

dtype('int64')

In [129]:
flights_2008.Diverted.dtype

dtype('int64')

In [130]:
flights_2008['Cancelled']=flights_2008.Cancelled.astype(bool)

In [131]:
flights_2008['Diverted']=flights_2008.Diverted.astype(bool)

## Test

In [132]:
flights_2008.Cancelled.dtype

dtype('bool')

In [133]:
flights_2008.Diverted.dtype

dtype('bool')

In [134]:
flights_2008[['Cancelled','Diverted']].head()

,Cancelled,Diverted
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False


## Code

We will create a new column Route of type object which is a Origin + “-“ + Dest.

In [135]:
flights_2008['Route'] = flights_2008.Origin +" - "+flights_2008.Dest

## Test

In [136]:
flights_2008.head()

,DayOfWeek,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,...,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDateTime,CRSDepDateTime,ArrDateTime,CRSArrDateTime,Route
0,4,WN,335,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,...,0.0,0.0,0.0,0.0,0.0,2008-01-03 20:03:00,2008-01-03 19:55:00,2008-01-03 22:11:00,2008-01-03 22:25:00,IAD - TPA
1,4,WN,3231,128.0,145.0,113.0,2.0,19.0,IAD,TPA,...,0.0,0.0,0.0,0.0,0.0,2008-01-03 07:54:00,2008-01-03 07:35:00,2008-01-03 10:02:00,2008-01-03 10:00:00,IAD - TPA
2,4,WN,448,96.0,90.0,76.0,14.0,8.0,IND,BWI,...,0.0,0.0,0.0,0.0,0.0,2008-01-03 06:28:00,2008-01-03 06:20:00,2008-01-03 08:04:00,2008-01-03 07:50:00,IND - BWI
3,4,WN,1746,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,...,0.0,0.0,0.0,0.0,0.0,2008-01-03 09:26:00,2008-01-03 09:30:00,2008-01-03 10:54:00,2008-01-03 11:00:00,IND - BWI
4,4,WN,3920,90.0,90.0,77.0,34.0,34.0,IND,BWI,...,2.0,0.0,0.0,0.0,32.0,2008-01-03 18:29:00,2008-01-03 17:55:00,2008-01-03 19:59:00,2008-01-03 19:25:00,IND - BWI


## Code

5. Change Carrier, Origin and Dest to Categorical.

In [137]:
flights_2008.UniqueCarrier.dtype,flights_2008.Origin.dtype,flights_2008.Dest.dtype

(dtype('O'), dtype('O'), dtype('O'))

In [138]:
flights_2008['UniqueCarrier']=flights_2008.UniqueCarrier.astype('category')
flights_2008['Origin']=flights_2008.Origin.astype('category')
flights_2008['Dest']=flights_2008.Dest.astype('category')

## Test

In [139]:
flights_2008.UniqueCarrier.dtype,flights_2008.Origin.dtype,flights_2008.Dest.dtype

(CategoricalDtype(categories=['9E', 'AA', 'AQ', 'AS', 'B6', 'CO', 'DL', 'EV', 'F9', 'FL',
                   'HA', 'MQ', 'NW', 'OH', 'OO', 'UA', 'US', 'WN', 'XE', 'YV'],
                  ordered=False),
 CategoricalDtype(categories=['ABE', 'ABI', 'ABQ', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY',
                   'ADK', 'ADQ',
                   ...
                   'TYR', 'TYS', 'VLD', 'VPS', 'WRG', 'WYS', 'XNA', 'YAK',
                   'YKM', 'YUM'],
                  ordered=False),
 CategoricalDtype(categories=['ABE', 'ABI', 'ABQ', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY',
                   'ADK', 'ADQ',
                   ...
                   'TYR', 'TYS', 'VLD', 'VPS', 'WRG', 'WYS', 'XNA', 'YAK',
                   'YKM', 'YUM'],
                  ordered=False))

## Code

We will create a new column with Boolean Delayed, which will populate if flight is delayed.

In [140]:
flights_2008['Delayed']=(flights_2008.CarrierDelay >0) |(flights_2008.WeatherDelay >0) | (flights_2008.NASDelay >0) |(flights_2008.SecurityDelay >0) |(flights_2008.LateAircraftDelay >0)

## Test

In [141]:
flights_2008[flights_2008.Delayed].head()

,DayOfWeek,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,...,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDateTime,CRSDepDateTime,ArrDateTime,CRSArrDateTime,Route,Delayed
4,4,WN,3920,90.0,90.0,77.0,34.0,34.0,IND,BWI,...,0.0,0.0,0.0,32.0,2008-01-03 18:29:00,2008-01-03 17:55:00,2008-01-03 19:59:00,2008-01-03 19:25:00,IND - BWI,True
6,4,WN,509,240.0,250.0,230.0,57.0,67.0,IND,LAS,...,0.0,0.0,0.0,47.0,2008-01-03 19:37:00,2008-01-03 18:30:00,2008-01-03 20:37:00,2008-01-03 19:40:00,IND - LAS,True
11,4,WN,1333,121.0,135.0,107.0,80.0,94.0,IND,MCO,...,0.0,0.0,0.0,72.0,2008-01-03 16:44:00,2008-01-03 15:10:00,2008-01-03 18:45:00,2008-01-03 17:25:00,IND - MCO,True
16,4,WN,675,228.0,240.0,213.0,15.0,27.0,IND,PHX,...,0.0,0.0,0.0,12.0,2008-01-03 14:52:00,2008-01-03 14:25:00,2008-01-03 16:40:00,2008-01-03 16:25:00,IND - PHX,True
18,4,WN,4,123.0,135.0,110.0,16.0,28.0,IND,TPA,...,0.0,0.0,0.0,16.0,2008-01-03 13:23:00,2008-01-03 12:55:00,2008-01-03 15:26:00,2008-01-03 15:10:00,IND - TPA,True


## Checkpoint and Storage

We have addressed below issues in data wrangling. Now we will store the dataframe into new CSV for Tableau.

1. Following columns have nan values:
    * DepTime – Missing for cancelled flights
    * ArrTime  – Missing for cancelled or diverted flights
    * ActualElapsedTime – Missing for cancelled or diverted flights
    * CRSElapsedTime – Missing for cancelled or diverted flights
    * AirTime  – Missing for cancelled or diverted flights
    * ArrDelay – Missing for cancelled or diverted flights
    * DepDelay – Missing for cancelled or diverted flights
    * TaxiIn – Missing for cancelled or diverted flights
    * TaxiOut – Missing for cancelled or diverted flights
    * CancellationCode – Missing for non-cancelled flight
    * CarrierDelay – Missing for non-delayed flights
    * WeatherDelay – Missing for non-delayed flights
    * NASDelay – Missing for non-delayed flights
    * SecurityDelay – Missing for non-delayed flights
    * LateAircraftDelay – Missing for non-delayed flights
2. Following columns are loaded as float64 instead of Time
    * DepTime
    * ArrTime
3. Following columns are loaded as int64 instead of Time
    * CRSDepTime
    * CRSArrTime
4. Following columns are loaded as int64 instead of Boolean
    * Cancelled
    * Diverted
5. Change Carrier, Origin and Dest to Categorical.
6. Year, Month and Day of Month are separate int64 attributes instead of Date. We will delete individual fields and create consolidated date time columns for CRSDep, CRSArr, Dep and Arr. There are some time fields with value 24:00 which should change date to next date.
7.	We will create a new column with Boolean Delayed, which will populate if flight is delayed.

    __Note__: In this case we cannot restructure delay time into one column because single delay can be caused by more than one reason.
8.	We will create a new column Route of type object which is a Origin + “-“ + Dest.
9. TailNum column will be deleted as we are not going to utilize this column in our analysis.

In [142]:
flights_2008.to_csv("flights_2008.csv",index=False)